<a href="https://colab.research.google.com/github/SBoo9/Ollama_Drive_setup/blob/main/Ollama_drive_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 1: Mount Google Drive
from google.colab import drive
import os
import subprocess
import time
import requests

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!pip install virtualenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 23.8 MB/s eta 0:00:00


In [3]:
!virtualenv /content/drive/MyDrive/colab_env

created virtual environment CPython3.11.12.final.0-64 in 23593ms
  creator CPython3Posix(dest=/content/drive/MyDrive/colab_env, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==25.1.1, setuptools==80.3.1
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [4]:
!source /content/drive/MyDrive/colab_env/bin/activate

In [5]:
# Step 2: Setup directories in Google Drive
DRIVE_PATH = "/content/drive/MyDrive"
OLLAMA_DIR = f"{DRIVE_PATH}/ollama_setup"
OLLAMA_BIN_DIR = f"{OLLAMA_DIR}/bin"
OLLAMA_MODELS_DIR = f"{OLLAMA_DIR}/models"

In [6]:
# Create directories if they don't exist
os.makedirs(OLLAMA_BIN_DIR, exist_ok=True)
os.makedirs(OLLAMA_MODELS_DIR, exist_ok=True)

In [7]:
print(f"Ollama directory: {OLLAMA_DIR}")
print(f"Models directory: {OLLAMA_MODELS_DIR}")

Ollama directory: /content/drive/MyDrive/ollama_setup
Models directory: /content/drive/MyDrive/ollama_setup/models


In [8]:
# Step 3: Download and install Ollama (only if not already present)
def install_ollama():
    ollama_binary = f"{OLLAMA_BIN_DIR}/ollama"

    if os.path.exists(ollama_binary):
        print("Ollama already installed in Google Drive!")
        return ollama_binary

    print("Installing Ollama...")

    # Download Ollama
    subprocess.run([
        "curl", "-fsSL", "https://ollama.com/install.sh"
    ], capture_output=False)

    # Download the binary directly
    subprocess.run([
        "curl", "-L", "https://github.com/ollama/ollama/releases/download/v0.1.48/ollama-linux-amd64",
        "-o", ollama_binary
    ], check=True)

    # Make it executable
    os.chmod(ollama_binary, 0o755)

    print("Ollama installed successfully!")
    return ollama_binary

In [9]:
ollama_path = install_ollama()

Installing Ollama...
Ollama installed successfully!


In [10]:
# Step 4: Setup environment variables
os.environ['OLLAMA_MODELS'] = OLLAMA_MODELS_DIR
os.environ['PATH'] = f"{OLLAMA_BIN_DIR}:{os.environ.get('PATH', '')}"

In [11]:
# Step 5: Start Ollama server
def start_ollama_server():
    print("Starting Ollama server...")

    # Kill any existing ollama processes
    subprocess.run(["pkill", "-f", "ollama"], capture_output=True)
    time.sleep(2)

    # Start ollama serve in background
    process = subprocess.Popen([
        ollama_path, "serve"
    ], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # Wait for server to start
    time.sleep(5)

    # Check if server is running
    try:
        response = requests.get("http://localhost:11434")
        if response.status_code == 200:
            print("✅ Ollama server started successfully!")
            return process
        else:
            print("❌ Server not responding properly")
            return None
    except requests.exceptions.RequestException:
        print("❌ Failed to connect to Ollama server")
        return None

In [12]:
server_process = start_ollama_server()

Starting Ollama server...
✅ Ollama server started successfully!


In [13]:
def list_available_models():
    """List models available for download"""
    popular_models = [
        "llama3.2:1b",      # Small, fast model
        "llama3.2:3b",      # Medium model
        "gemma2:2b",        # Google's Gemma
        "phi3:mini",        # Microsoft's Phi-3
        "mistral:7b",       # Mistral 7B
        "codellama:7b",     # Code-focused model
    ]

    print("Popular models you can download:")
    for model in popular_models:
        print(f"  - {model}")

    return popular_models

In [14]:
def download_model(model_name):
    """Download a model to persistent storage"""
    print(f"Downloading {model_name}...")

    result = subprocess.run([
        ollama_path, "pull", model_name
    ], capture_output=True, text=True)

    if result.returncode == 0:
        print(f"✅ {model_name} downloaded successfully!")
        print(f"Model stored in: {OLLAMA_MODELS_DIR}")
    else:
        print(f"❌ Failed to download {model_name}")
        print(f"Error: {result.stderr}")

In [15]:
def list_installed_models():
    """List models already downloaded"""
    result = subprocess.run([
        ollama_path, "list"
    ], capture_output=True, text=True)

    if result.returncode == 0:
        print("Installed models:")
        print(result.stdout)
    else:
        print("No models installed yet")

In [16]:
def chat_with_model(model_name, prompt):
    """Chat with a model"""
    result = subprocess.run([
        ollama_path, "run", model_name, prompt
    ], capture_output=True, text=True)

    if result.returncode == 0:
        return result.stdout.strip()
    else:
        return f"Error: {result.stderr}"

In [17]:
# Step 7: Test the setup
print("\n" + "="*50)
print("OLLAMA SETUP COMPLETE!")
print("="*50)

# List available models
list_available_models()


OLLAMA SETUP COMPLETE!
Popular models you can download:
  - llama3.2:1b
  - llama3.2:3b
  - gemma2:2b
  - phi3:mini
  - mistral:7b
  - codellama:7b


['llama3.2:1b',
 'llama3.2:3b',
 'gemma2:2b',
 'phi3:mini',
 'mistral:7b',
 'codellama:7b']

In [18]:
print("\nChecking installed models...")
list_installed_models()


Checking installed models...
Installed models:
NAME	ID	SIZE	MODIFIED 



In [19]:
print("\n" + "="*50)
print("USAGE INSTRUCTIONS:")
print("="*50)
print("1. Download a model: download_model('llama3.2:1b')")
print("2. Chat with model: chat_with_model('llama3.2:1b', 'Hello!')")
print("3. List models: list_installed_models()")
print("4. Your models are saved in Google Drive and will persist!")


USAGE INSTRUCTIONS:
1. Download a model: download_model('llama3.2:1b')
2. Chat with model: chat_with_model('llama3.2:1b', 'Hello!')
3. List models: list_installed_models()
4. Your models are saved in Google Drive and will persist!


In [20]:
# Example usage
print("\n" + "="*30)
print("EXAMPLE USAGE:")
print("="*30)
print("""
# Download a small model (recommended for testing)
download_model('llama3.2:1b')

# Chat with the model
response = chat_with_model('llama3.2:1b', 'Explain what is machine learning in simple terms')
print(response)

# For your medical RAG project, you might want:
download_model('llama3.2:3b')  # Better performance for complex medical queries
""")


EXAMPLE USAGE:

# Download a small model (recommended for testing)
download_model('llama3.2:1b')

# Chat with the model
response = chat_with_model('llama3.2:1b', 'Explain what is machine learning in simple terms')
print(response)

# For your medical RAG project, you might want:
download_model('llama3.2:3b')  # Better performance for complex medical queries



In [21]:
def setup_rag_environment():
    """Install additional packages for RAG"""
    packages = [
        "langchain",
        "chromadb",
        "sentence-transformers",
        "transformers",
        "Pillow",
        "easyocr",
        "pypdf"
    ]

    for package in packages:
        print(f"Installing {package}...")
        subprocess.run(["pip", "install", package], capture_output=True)

    print("✅ RAG environment setup complete!")

# Uncomment the next line to install RAG packages
# setup_rag_environment()


In [22]:
print("\nYour Ollama setup is now persistent in Google Drive!")
print(f"Location: {OLLAMA_DIR}")
print("Next time you open Colab, just run the startup section to reconnect!")


Your Ollama setup is now persistent in Google Drive!
Location: /content/drive/MyDrive/ollama_setup
Next time you open Colab, just run the startup section to reconnect!
